In [ ]:
!pip install transformers datasets torch

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 788.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.9/447.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("GBaker/MedQA-USMLE-4-options", split='test')

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
def get_model_prediction(question, options_dict):
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"Question: {question}\nOptions: {options_text}\nAnswer:"

    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    output = model.generate(inputs["input_ids"], max_new_tokens=172)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    match = re.search(r"Answer:\s*([A-D])", generated_text)

    if match:
        predicted_option = match.group(1)
    else:
        predicted_option = None

    return predicted_option

In [ ]:
correct_predictions = 0
total_predictions = 0
batch_size = 200


for i in range(601, len(dataset), batch_size):
    batch = dataset.select(range(i, min(i + batch_size, len(dataset))))

    for sample in tqdm(batch, desc=f'Processing batch {i//batch_size + 1}'):
        question = sample['question']
        options = sample['options']
        correct_answer = sample['answer_idx']

        predicted_answer = get_model_prediction(question, options)

        if predicted_answer == correct_answer:
            correct_predictions += 1

        total_predictions += 1
        print(f"Accuracy : {correct_predictions / total_predictions * 100:.2f}%")

    print(f"Accuracy after batch {i//batch_size + 1}: {correct_predictions / total_predictions * 100:.2f}%")

Processing batch 4:   0%|          | 0/200 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.
Processing batch 4:   0%|          | 1/200 [00:29<1:36:11, 29.00s/it]

Accuracy : 100.00%


Processing batch 4:   1%|          | 2/200 [00:58<1:35:42, 29.00s/it]

Accuracy : 50.00%


Processing batch 4:   2%|▏         | 3/200 [01:25<1:32:21, 28.13s/it]

...

Accuracy : 11.62%


Processing batch 5:  64%|██████▍   | 128/200 [1:27:33<56:36, 47.17s/it]

Accuracy : 11.59%


Processing batch 5:  64%|██████▍   | 129/200 [1:28:21<56:15, 47.54s/it]

Accuracy : 11.55%


In [1]:
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 14.52%
